In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [4]:
df = pd.read_excel('streams/AEStreamTable2.xlsx')
df

,Stream Number,Type,Molar Flow Rate [kmol/hr],Tin,Tout,T_diff,Cp (kJ/kmol/K),Duty [kJ/hr],Duty [kW],Flux [kW / K]
0,S2,cold,162.598652,93.790304,150.000000,-56.209696,175.453272,-1.603576e+06,-445.437877,7.924574
1,S9,cold,309.317662,124.850263,619.999892,-495.149629,226.374457,-3.467118e+07,-9630.882811,19.450449
2,S11,cold,256.000000,20.000000,620.000000,-600.000000,29.584853,-4.544233e+06,-1262.287059,2.103812
3,S12,hot,575.347324,620.000000,2.000000,618.000000,124.000000,4.409002e+07,12250.000000,19.822006
4,S15,cold,158.352029,0.547123,150.000000,-149.452877,150.560459,-3.563189e+06,-989.774681,6.622654
5,S21,cold,500.000000,20.000000,150.000000,-130.000000,29.089416,-1.890812e+06,-525.225564,4.040197
6,S22,hot,337.018246,150.000000,25.000000,125.000000,139.855395,5.891727e+06,1636.590970,13.092728
7,S24,hot,322.000000,25.000000,23.100976,1.899024,149.396949,9.135412e+04,25.388889,13.369439
8,S25,cold,7.497616,24.999994,150.000000,-125.000006,29.392114,-2.754635e+04,-7.651764,0.061214
9,S27,cold,215.715500,20.000000,100.000000,-80.000000,199.685133,-3.446014e+06,-957.226184,11.965327


In [5]:
print(df)

   Stream Number  Type  Molar Flow Rate [kmol/hr]         Tin        Tout  \
0             S2  cold                 162.598652   93.790304  150.000000   
1             S9  cold                 309.317662  124.850263  619.999892   
2            S11  cold                 256.000000   20.000000  620.000000   
3            S12   hot                 575.347324  620.000000    2.000000   
4            S15  cold                 158.352029    0.547123  150.000000   
5            S21  cold                 500.000000   20.000000  150.000000   
6            S22   hot                 337.018246  150.000000   25.000000   
7            S24   hot                 322.000000   25.000000   23.100976   
8            S25  cold                   7.497616   24.999994  150.000000   
9            S27  cold                 215.715500   20.000000  100.000000   
10           S28  cold                 196.104942   56.389812   96.018129   
11           S29  cold                 125.918081   61.796286   80.000000   

In [6]:
df['Stream Number']

0      S2
1      S9
2     S11
3     S12
4     S15
5     S21
6     S22
7     S24
8     S25
9     S27
10    S28
11    S29
12    S32
13    S37
Name: Stream Number, dtype: object

## Net utility usage per interval

We have 20 intervals: $Q_1, Q_2, \dots , Q_{20}$

Let flux of stream i be denoted as $Fl_i$ $[\frac{kW}{K}]$


Let the temperature change in interval $Q_j$ be $\Delta T_{Q_j}$ $[K]$

$
Q_j = \sum_{i = 1}^{14} Fl_i \cdot \Delta T_{Q_j} \cdot \delta_{i, j} 
$


where

$
\delta_{i, j} = 
\left\{
    \begin{array}{lr}
        1, & \text{if } i \in Q_j \\
        0, & \text{if } i \notin Q_j 
    \end{array}
\right\}
$

In [7]:
# Number of streams and intervals
num_streams = 14
num_intervals = 20

# Initialize the delta matrix with zeros
delta_matrix = np.zeros((num_intervals, num_streams))

# Streams in each interval where for example: 1: [9, 11] implies that streams 9 and 11 are in interval 1
streams_in_intervals = {
    1: ['S9', 'S11'],
    2: ['S12', 'S9', 'S11'],
    3: ['S12', 'S2', 'S9', 'S11', 'S15', 'S21', 'S25'],
    4: ['S22', 'S12', 'S2', 'S9', 'S11', 'S15', 'S21', 'S25'],
    5: ['S22', 'S12', 'S2', 'S11', 'S15', 'S21', 'S25'],
    6: ['S22', 'S12', 'S2', 'S11', 'S15', 'S21', 'S25', 'S27'],
    7: ['S22', 'S12', 'S2', 'S28', 'S11', 'S15', 'S21', 'S25', 'S27'],
    8: ['S22', 'S12', 'S2', 'S28', 'S11', 'S15', 'S21', 'S25', 'S27'],
    9: ['S37', 'S22', 'S12', 'S28', 'S11', 'S15', 'S21', 'S25', 'S27'],
    10: ['S37', 'S22', 'S12', 'S28', 'S29', 'S11', 'S15', 'S21', 'S25', 'S27'],
    11: ['S37', 'S22', 'S12', 'S28', 'S29', 'S11', 'S15', 'S21', 'S25', 'S27', 'S32'],
    12: ['S22', 'S12', 'S28', 'S29', 'S11', 'S15', 'S21', 'S25', 'S27', 'S32'],
    13: ['S22', 'S12', 'S28', 'S11', 'S15', 'S21', 'S25', 'S27', 'S32'],
    14: ['S22', 'S12', 'S28', 'S11', 'S15', 'S21', 'S25', 'S27', 'S32'],
    15: ['S22', 'S12', 'S11', 'S15', 'S21', 'S27', 'S32'],
    16: ['S22', 'S12', 'S15', 'S32'],
    17: ['S22', 'S12', 'S15'],
    18: ['S24', 'S12', 'S15'],
    19: ['S12', 'S15'],
    20: ['S12'],
}



In [8]:
# Create a mapping from stream names to their indices
stream_to_index = {stream: idx for idx, stream in enumerate(df['Stream Number'])}

# Function to replace stream names with their indices
def replace_streams_with_indices(streams_in_intervals, stream_to_index):
    new_intervals = {}
    for interval, streams in streams_in_intervals.items():
        new_intervals[interval] = [stream_to_index.get(str(stream), stream) for stream in streams]
    return new_intervals

# Use the function to create the new table
new_streams_in_intervals = replace_streams_with_indices(streams_in_intervals, stream_to_index)

# print(new_streams_in_intervals)

In [9]:
temp_differences_intervals = {
    1: 10, 
    2: 60, 
    3: 10, 
    4: 15, 
    5: 25, 
    6: 4, 
    7: 2, 
    8: 8, 
    9: 2, 
    10: 4, 
    11: 10, 
    12: 8, 
    13: 6, 
    14: 31, 
    15: 5,
    16: 4, 
    17: 1, 
    18: 2, 
    19: 12.45, 
    20: 8.55
}

In [10]:
df['Type'][1]

'cold'

In [21]:
def calculate_Qj(fluxes, temp_changes, streams_in_intervals):
    Qj_values = {}
    for interval, streams in streams_in_intervals.items():
        total_heat_transfer = 0
        for stream_number in streams:
            # Check if stream_number exists in both fluxes and temp_changes
            if stream_number in fluxes and stream_number in temp_changes:
#                 print(df['Type'][stream_number])
#                 print(stream_number)   
                if df['Type'][stream_number] == 'cold':
                    total_heat_transfer -= fluxes[stream_number] * temp_changes[stream_number]
                else: 
                    total_heat_transfer += fluxes[stream_number] * temp_changes[stream_number]
        Qj_values[interval] = total_heat_transfer
    return Qj_values


In [22]:
# Calculate Qj values
Qj_results = calculate_Qj(df['Flux [kW / K]'], temp_differences_intervals, new_streams_in_intervals)
print(Qj_results)


{1: -320.73319990962216, 2: -122.51313518470309, 3: -323.3475732709486, 4: -270.9766622405486, 5: -76.4721682209264, 6: -100.4028228111764, 7: -124.36498497865281, 8: -124.36498497865281, 9: -99.73020305793395, 10: -160.95298145035645, 11: -170.68878916053484, 12: -195.3235710812537, 13: -134.1007926888312, 14: -134.1007926888312, 15: -109.64891765978157, 16: 141.51535895316073, 17: 151.25116666333912, 18: 125.61913280823971, 19: 98.88025563293913, 20: 198.2200647249191}


## Net utility usage per interval

In [23]:
df1 = pd.DataFrame(Qj_results, index=[0])

In [24]:
df1

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20
0,-320.7332,-122.513135,-323.347573,-270.976662,-76.472168,-100.402823,-124.364985,-124.364985,-99.730203,-160.952981,-170.688789,-195.323571,-134.100793,-134.100793,-109.648918,141.515359,151.251167,125.619133,98.880256,198.220065


Casade heat ? 
